# The Unreasonable Effectiveness of Linear Prediction as a Perceptual Metric

- https://github.com/dsevero/Linear-Autoregressive-Similarity-Index
- https://github.com/Na-moe/LASI-PyTorch
- https://arxiv.org/abs/2310.05986


# Setup (run this first)

## Clone repo

In [ ]:
!git clone https://github.com/Na-moe/LASI-PyTorch.git ./lasi_pytorch
%cd lasi_pytorch

## Install dependencies

In [1]:
!./install.sh

Looking in indexes: https://download.pytorch.org/whl/cu121
Obtaining file:///home/namoe/lasi_pytorch
  Preparing metadata (setup.py) ... done
  Attempting uninstall: lasi_pytorch
    Found existing installation: lasi_pytorch 0.0.0
    Uninstalling lasi_pytorch-0.0.0:
      Successfully uninstalled lasi_pytorch-0.0.0
  Running setup.py develop for lasi_pytorch


# How to use LASI-PyTorch in your own code

In [1]:
%load_ext autoreload
%autoreload 2

In [28]:
# you may need to install pillow first
from PIL import Image
from lasi_pytorch import LASI
import torch
import time

import numpy as np

# set cuda device
torch.set_default_device('cuda')

# Load images.
img_megg = torch.tensor(np.array(Image.open('assets/megg.png').convert('RGB')))
img_dark_megg = torch.tensor(np.array(Image.open('assets/dark-megg.png').convert('RGB')))
assert img_dark_megg.shape == img_megg.shape

# Compute the distance between img_megg and img_dark_megg.
lasi = LASI(img_megg.shape, neighborhood_size=10, use_vmap=True)
print(lasi.use_vmap)
lasi = torch.compile(lasi)
start = time.time()
for i in range(100):
    distance = lasi.compute_distance(img_megg, img_dark_megg)
# sync
torch.cuda.synchronize()
print(f'Elapsed time: {time.time() - start:.3f} s')
print(f'd(img_megg, img_dark_megg) = {distance}')

# Efficiently compute the distance between multiple images relative to a reference (img_megg).
# This function jits internally.
img_megg_offset = torch.clip(img_megg + 20, 0 ,255)
distances = lasi.compute_distance_multiple(
    ref=img_megg, p0=img_dark_megg, p1=img_megg_offset)
print(f"d(ref, p0) = {distances['p0']}")
print(f"d(ref, p1) = {distances['p1']}")

lasi0 = LASI(img_megg.shape, neighborhood_size=10, use_vmap=False)
print(lasi0.use_vmap)
lasi0 = torch.compile(lasi0)
start = time.time()
for i in range(100):
    distance0 = lasi0.compute_distance(img_megg, img_dark_megg)
torch.cuda.synchronize()
print(f'Elapsed time: {time.time() - start:.3f} s')
print(f'd(img_megg, img_dark_megg) = {distance0}')
distances0 = lasi0.compute_distance_multiple(
    ref=img_megg, p0=img_dark_megg, p1=img_megg_offset)
print(f"d(ref, p0) = {distances0['p0']}")
print(f"d(ref, p1) = {distances0['p1']}")

True
Elapsed time: 16.053 s
d(img_megg, img_dark_megg) = 1.3687046766281128
d(ref, p0) = 1.3687046766281128
d(ref, p1) = 1.349355697631836
False
Elapsed time: 9.634 s
d(img_megg, img_dark_megg) = 1.3687046766281128
d(ref, p0) = 1.3687046766281128
d(ref, p1) = 1.349355697631836
